In [34]:
%matplotlib widget

import logging
import time

import cftime
import httpx
import matplotlib.pyplot as plt
import netCDF4
import pandas as pd
import shapely.io
import sqlmodel
from sqlalchemy import (
    bindparam, 
    func
)

from arpav_ppcv import (
    database as db,
    operations,
)
from arpav_ppcv.config import get_settings
from arpav_ppcv.schemas.base import (
    CoverageDataSmoothingStrategy,
    ObservationDataSmoothingStrategy,
    ObservationAggregationType,
    Season,
)
from arpav_ppcv.schemas import coverages
from arpav_ppcv.schemas.coverages import CoverageInternal

logging.basicConfig(level=logging.DEBUG)
logging.getLogger("httpx").setLevel(logging.WARNING)
logging.getLogger("httpcore").setLevel(logging.WARNING)
logging.getLogger("matplotlib").setLevel(logging.WARNING)

settings = get_settings()
session = sqlmodel.Session(db.get_engine(settings))
http_client = httpx.Client()

In [2]:
coverage_identifier = "tas_seasonal_absolute_model_ensemble-model_ensemble-absolute-rcp26-DJF"
point_coords = "POINT(11.5469 44.9524)"
date_range = "../.."

cov = CoverageInternal(
    configuration=db.get_coverage_configuration_by_coverage_identifier(session, coverage_identifier), 
    identifier=coverage_identifier
)

In [29]:
pv_cte = (
     sqlmodel
            .select(
                coverages.CoverageConfiguration.id,
                func.jsonb_agg(
                    func.json_build_object(
                        coverages.ConfigurationParameter.name,
                        coverages.ConfigurationParameterValue.name,
                    )
                ).label("possible_values")
            )
            .join(
                coverages.ConfigurationParameterPossibleValue,
                coverages.CoverageConfiguration.id ==
                coverages.ConfigurationParameterPossibleValue.coverage_configuration_id
            )
            .join(
                coverages.ConfigurationParameterValue,
                coverages.ConfigurationParameterValue.id ==
                coverages.ConfigurationParameterPossibleValue.configuration_parameter_value_id
            )
            .join(
                coverages.ConfigurationParameter,
                coverages.ConfigurationParameter.id ==
                coverages.ConfigurationParameterValue.configuration_parameter_id
            ).group_by(coverages.CoverageConfiguration.id)
).cte("cov_conf_possible_values")

In [30]:
print(pv_cte)

SELECT coverageconfiguration.id, jsonb_agg(json_build_object(configurationparameter.name, configurationparametervalue.name)) AS possible_values 
FROM coverageconfiguration JOIN configurationparameterpossiblevalue ON coverageconfiguration.id = configurationparameterpossiblevalue.coverage_configuration_id JOIN configurationparametervalue ON configurationparametervalue.id = configurationparameterpossiblevalue.configuration_parameter_value_id JOIN configurationparameter ON configurationparameter.id = configurationparametervalue.configuration_parameter_id GROUP BY coverageconfiguration.id


In [39]:
pvs = {
    "climatological_model": "model_ensemble",
    "scenario": "rcp26"
}

In [38]:
statement = (
    sqlmodel.select(coverages.CoverageConfiguration)
    .join(pv_cte, pv_cte.c.id == coverages.CoverageConfiguration.id)
    .where(func.jsonb_path_exists(pv_cte.c.possible_values, '$[*] ? (@.climatological_model == "model_ensemble")'))
    .where(func.jsonb_path_exists(pv_cte.c.possible_values, '$[*] ? (@.scenario == "rcp26")'))
)

In [40]:
statement = (
    sqlmodel.select(coverages.CoverageConfiguration)
    .join(pv_cte, pv_cte.c.id == coverages.CoverageConfiguration.id)
)

In [41]:
for value_name, value in pvs.items():
    statement = (
        statement
        .where(func.jsonb_path_exists(pv_cte.c.possible_values, f'$[*] ? (@.{value_name} == "{value}")'))
    )

In [42]:
print(statement.compile(db.get_engine(settings), compile_kwargs={"literal_binds": True}))

WITH cov_conf_possible_values AS 
(SELECT coverageconfiguration.id AS id, jsonb_agg(json_build_object(configurationparameter.name, configurationparametervalue.name)) AS possible_values 
FROM coverageconfiguration JOIN configurationparameterpossiblevalue ON coverageconfiguration.id = configurationparameterpossiblevalue.coverage_configuration_id JOIN configurationparametervalue ON configurationparametervalue.id = configurationparameterpossiblevalue.configuration_parameter_value_id JOIN configurationparameter ON configurationparameter.id = configurationparametervalue.configuration_parameter_id GROUP BY coverageconfiguration.id)
 SELECT coverageconfiguration.id, coverageconfiguration.name, coverageconfiguration.display_name_english, coverageconfiguration.display_name_italian, coverageconfiguration.description_english, coverageconfiguration.description_italian, coverageconfiguration.netcdf_main_dataset_name, coverageconfiguration.thredds_url_pattern, coverageconfiguration.wms_main_layer_nam

In [43]:
session.rollback()

In [44]:
session.exec(statement).all()

[CoverageConfiguration(description_english='TAS seasonal absolute model ensemble', color_scale_max=32.0, description_italian='TAS valore assoluto di stagione media ensemble', observation_variable_id=UUID('32706ce0-0134-4122-91a0-9d6f237350b3'), netcdf_main_dataset_name='tas', observation_variable_aggregation_type=<ObservationAggregationType.SEASONAL: 'SEASONAL'>, thredds_url_pattern='ensymbc/clipped/tas_avg_{scenario}_{year_period}_ts19762100_ls_VFVG.nc', uncertainty_lower_bounds_coverage_configuration_id=UUID('9c2bfa7a-d108-4d2d-8c99-f26ae9329e79'), name='tas_seasonal_absolute_model_ensemble', wms_main_layer_name='tas', uncertainty_upper_bounds_coverage_configuration_id=UUID('d33d6b43-0379-447a-ad9d-b2f36f7dce3e'), id=UUID('06c104d2-a502-400b-9728-b8e2d9ee65d9'), unit='ºC', display_name_english='TAS seasonal absolute model ensemble', palette='default/seq-YlOrRd', display_name_italian='TAS valore assoluto di stagione media ensemble', color_scale_min=-3.0, coverage_id_pattern='{name}-{c